In [44]:
import copy
import pickle
import re
from typing import List
import tqdm

import numpy as np

In [20]:
import openff.interchange
import openff.toolkit

from openff.interchange.components.interchange import Interchange
from openff.toolkit.topology import Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

import openmm
import pandas as pd

In [3]:
print(openff.interchange.__version__)

v0.2.0-alpha.5


In [4]:
print(openff.toolkit.__version__)

0.10.1+102.g57a442b5


In [5]:
from interchange_regression_utilities.models import (
    ComparisonSettings,
    ExpectedValueChange,
    model_from_file,
    TopologyDefinition,
)

In [6]:
models = model_from_file(List[TopologyDefinition], "input-topologies.json")

In [10]:
topologies = []
failed_smiles = []
smiles_pattern = "SMILES '([A-Za-z0-9=\(\)\[\]]+)'"
for model in tqdm.tqdm(models):
    try:
        topologies.append(model.to_topology())
    except ValueError as e:
        if "No registered toolkits can provide the capability" in str(e):
            smiles = re.search(smiles_pattern, str(e)).group(1)
            failed_smiles.append(smiles)

 51%|█████████████████████▉                     | 26/51 [08:11<10:04, 24.20s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 15, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral:

In [15]:
len(failed_smiles)

0

In [16]:
topology_file = "openff_topologies.pkl"
with open(topology_file, "wb") as f:
    pickle.dump(topologies, f)
# with open(topology_file, "rb") as f:
#     topologies = pickle.load(f)

In [17]:
force_field = ForceField("openff-2.0.0.offxml")

In [18]:
interchanges = [
    Interchange.from_smirnoff(force_field, topology)
    for topology in tqdm.tqdm(topologies)
]

interchange_systems = [
    ichange.to_openmm(combine_nonbonded_forces=True)
    for ichange in interchanges
]

100%|███████████████████████████████████████████| 51/51 [00:09<00:00,  5.20it/s]


In [19]:
toolkit_systems = [
    force_field.create_openmm_system(topology, use_interchange=False)
    for topology in tqdm.tqdm(topologies)
]

 51%|█████████████████████▉                     | 26/51 [08:27<10:39, 25.57s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 4, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 5, aromatic: True, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 15, aromatic: False, chiral: False

 67%|████████████████████████████▋              | 34/51 [10:30<04:03, 14.34s/it]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 7, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 6, aromatic: True, chiral: False
bond order: 

In [24]:
def get_charges(
    system,
    model_index: int = 0,
    version: str = "interchange",
) -> pd.DataFrame:
    force = [
        force for force in system.getForces()
        if "NonbondedForce" in str(type(force))
    ][0]
    
    name = models[model_index].name
    data = {"particle_index": [], "charge": []}
    for i in range(force.getNumParticles()):
        charge, *_ = force.getParticleParameters(i)
        data["particle_index"].append(i)
        data["charge"].append(charge._value)
    
    df = pd.DataFrame(data)
    df["name"] = name
    df["version"] = version
    return df

In [54]:
all_charge_dfs = []

for i, (ichange, toolkit) in tqdm.tqdm(
    list(enumerate(zip(interchange_systems, toolkit_systems)))
):
    model = models[i]
    
    name = model.name
    ixml = openmm.XmlSerializer.serialize(ichange)
    txml = openmm.XmlSerializer.serialize(toolkit)
    
    ichange_name = f"interchange-{openff.interchange.__version__}"
    toolkit_name = f"toolkit-{openff.toolkit.__version__}"
    ifile = f"xmls/{name}_{ichange_name}.xml"
    tfile = f"xmls/{name}_{toolkit_name}.xml"
    
    with open(ifile, "w") as f:
        f.write(ixml)
    with open(tfile, "w") as f:
        f.write(txml)
        
    idf = get_charges(ichange, model_index=i, version=ichange_name)
    tdf = get_charges(toolkit, model_index=i, version=toolkit_name)
    
    model_df = copy.deepcopy(tdf)
    model_df["version"] = "original_topology"
    model_df["charge"] = [
        charge
        for molecule in topologies[i].topology_molecules
        for charge in np.array(molecule.partial_charges)
    ]
    
#     topology2 = model.to_topology()
#     model_df2 = pd.DataFrame(tdf)
#     model_df2["version"] = "new_topology"
#     model_df2["charge"] = [
#         charge._value
#         for molecule in topologies[i].topology_molecules
#         for charge in molecule.partial_charges
#     ]
    
    
    all_charge_dfs.extend(
        [idf, tdf, model_df,
#          model_df2,
    ])

charge_df = pd.concat(all_charge_dfs)

  0%|                                                    | 0/51 [00:00<?, ?it/s]/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topol

/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/v

/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/var/folders/rv/j6lbln6j0kvb5svxj8wflc400000gn/T/ipykernel_40801/602003120.py:30: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  for charge in np.array(molecule.partial_charges)
/Users/lily/pydev/toolkit/openff/toolkit/topology/topology.py:51: TopologyDeprecationWarning: Topology.topology_molecules is deprecated. Use Topology.molecules instead.
  warnings.warn(
/v

In [55]:
charge_df.version.unique()

array(['interchange-v0.2.0-alpha.5', 'toolkit-0.10.1+102.g57a442b5',
       'original_topology'], dtype=object)

In [56]:
ichange_df = charge_df[charge_df["version"] == "interchange-v0.2.0-alpha.5"]
toolkit_df = charge_df[charge_df["version"] == "toolkit-0.10.1+102.g57a442b5"]
topology_df = charge_df[charge_df["version"] == "original_topology"]

In [50]:
max(ichange_df.charge - toolkit_df.charge)

0.0

In [52]:
max(topology_df.charge - topology_df.charge)

0.0

In [68]:
charge_df.to_csv("charges_v011.csv")

In [58]:
older = pd.read_csv("charges_v010.csv", index_col=0)

In [59]:
combined = pd.concat([charge_df, older])

In [60]:
combined

,particle_index,charge,name,version
0,0,-0.91150,0,interchange-v0.2.0-alpha.5
1,1,0.16355,0,interchange-v0.2.0-alpha.5
2,2,0.02260,0,interchange-v0.2.0-alpha.5
3,3,-0.08142,0,interchange-v0.2.0-alpha.5
4,4,-0.10215,0,interchange-v0.2.0-alpha.5
...,...,...,...,...
45,45,0.15917,50,original_topology_v0.10
46,46,0.15954,50,original_topology_v0.10
47,47,0.03068,50,original_topology_v0.10
48,48,0.07602,50,original_topology_v0.10


In [61]:
ichange_df = combined[combined["version"] == "interchange-v0.2.0-alpha.5"]
toolkit_df = combined[combined["version"] == "toolkit-0.10.1+102.g57a442b5"]
topology_df = combined[combined["version"] == "original_topology"]
old_ichange_df = combined[combined["version"] == "interchange-v0.1.4"]
old_topology_df = combined[combined["version"] == "original_topology_v0.10"]

In [63]:
max(ichange_df.charge.values - old_ichange_df.charge.values)

0.052609994395170334

In [64]:
max(topology_df.charge.values - old_topology_df.charge.values)

2.220446049250313e-16

In [65]:
max(ichange_df.charge.values - toolkit_df.charge.values)

0.0

In [66]:
max(ichange_df.charge.values - topology_df.charge.values)

0.0

In [67]:
max(ichange_df.charge.values - old_topology_df.charge.values)

2.220446049250313e-16